# AOC2019 - Day 11

Today's challenge: [Day 11](https://adventofcode.com/2019/day/11)

Data are manually downloaded into `./data/input_11.txt`

In [1]:
with open('./data/input_11.txt') as f:
    raw = f.read()

In [2]:
class Intcode:
    def __init__(self, xs, pos=0, i=None):
        self.xs = xs
        self.pos = pos
        self.rel_base = 0
        self.i = i
        self.res = 0
        
    def display(self):
        return dict(enumerate(self.xs))
    
    @staticmethod
    def parse_op(x):
        params, op = divmod(x, 100)
        params, p1 = divmod(params, 10)
        params, p2 = divmod(params, 10)
        p3 = params % 10
        return op, p1, p2, p3
    
    def get_value(self,i,p):
        if p == 0:
            return self.xs[i]   
        elif p == 1:
            return i
        elif p == 2:
            i_ = i + self.rel_base
            return self.xs[i_]
        else:
            print("WTF:", p, i)
    
    def set_value(self,i,p):
        if p == 0:
            return i
        elif p == 2:
            i_ = i + self.rel_base
            return i_
        else:
            print("WTF:", p, i)
    
    def op1(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = i1_ + i2_
        return 4
    
    def op2(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = i1_ * i2_
        return 4

    def op3(self,x,i,ps):
        p, *_ = ps
        i_ = self.set_value(i, p)
        self.xs[i_] = x
        return 2

    def op4(self,i,ps):
        p, *_ = ps
        i_ = self.get_value(i, p)
        self.res = i_
        return 2
    
    def op5(self,i,out,ps):
        pi, pout, _ = ps
        i_ = self.get_value(i, pi)
        #out_ = self.set_value(out, pout)
        out_ = self.get_value(out, pout)
        if i_ != 0:
            self.pos = out_
            return 0
        return 3
    
    def op6(self,i,out,ps):
        pi, pout, _ = ps
        i_ = self.get_value(i, pi)
        #out_ = self.set_value(out, pout)
        out_ = self.get_value(out, pout)
        if i_ == 0:
            self.pos = out_
            return 0
        return 3
    
    def op7(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = 1 if i1_ < i2_ else 0
        return 4
    
    def op8(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = 1 if i1_ == i2_ else 0
        return 4
    
    def op9(self,i,ps):
        p, *_ = ps
        i_ = self.get_value(i, p)
        self.rel_base += i_
        return 2
    
    def op99(self):
        return 0

    def run(self, x=0):
        while True:
            op, *params = self.parse_op(self.xs[self.pos])
            if op%100 == 1:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op1(i1, i2, out, params)
            elif op%100 == 2:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op2(i1, i2, out, params)
            elif op%100 == 3:
                i = self.xs[self.pos+1]
                self.pos += self.op3(x, i, params)
            elif op%100 == 4:
                i = self.xs[self.pos+1]
                self.pos += self.op4(i, params)
                return self.res, True
            elif op%100 == 5:
                i, out = [self.xs[x] for x in range(self.pos+1,self.pos+3)]
                p = self.op5(i, out, params)
                self.pos += p
            elif op%100 == 6:
                i, out = [self.xs[x] for x in range(self.pos+1,self.pos+3)]
                p = self.op6(i, out, params)
                self.pos += p
            elif op%100 == 7:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op7(i1, i2, out, params)
            elif op%100 == 8:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op8(i1, i2, out, params)
            elif op%100 == 9:
                i1 = self.xs[self.pos+1]
                self.pos += self.op9(i1, params)
            elif op%100 == 99:
                self.op99()
                return self.res, False
            else:
                print("invalid instruction!", op)
                break

In [93]:
class Robot:
    def __init__(self, brain):
        self.p = brain
        self.panel = defaultdict(lambda : [0])
        self.directions = ['u','r','d','l']
        self.curr_d = 0
        self.pos_x = 0
        self.pos_y = 0
        
    def __repr__(self):
        xs, ys = zip(*self.panel.keys())
        x0, x1 = min(xs), max(xs)
        y0, y1 = min(ys), max(ys)
        res = []
        for y in range(x0, x1+1):
            row = ""
            for x in range(0,75):
                if self.panel[(x,y)][-1] == 1:
                    row += '#'
                else:
                    row += ' '
            res.append(row)
        return "\n".join(res[::-1])
        
    def rotate(self, d):
        if d == 1:
            self.curr_d += 1
        elif d == 0:
            self.curr_d += 3
        self.curr_d %= 4
        
    def move(self):
        d = self.directions[self.curr_d] 
        if d == 'u':
            self.pos_y += 1
        elif d == 'd':
            self.pos_y -= 1
        elif d == 'r':
            self.pos_x += 1
        elif d == 'l':
            self.pos_x -= 1
            
    def color(self, c):
        self.panel[(self.pos_x, self.pos_y)].append(c)
        
    def run(self):
        c = True
        while c:
            current_tile = self.panel[(self.pos_x, self.pos_y)][-1]
            #print("Now in (%s,%s), color is %s, facing %s" % (self.pos_x, self.pos_y, current_tile, self.curr_d))
            out1, _ = self.p.run(current_tile)
            out2, c = self.p.run(current_tile)
            #print("Now moving %s and painting %s" % (out1, out2))
            self.color(out1)
            self.rotate(out2)
            self.move()

In [94]:
from collections import defaultdict
data = defaultdict(int)
data.update({(i,int(x)) for i,x in enumerate(raw.strip().split(","))})
p = Intcode(data)
r = Robot(p)
r.panel[(0,0)] = [1]
r.run()
r

  ##  #  #  ##  #  # #### #### ###  #  #                                   
 #  # #  # #  # #  #    # #    #  # # #                                    
 #  # #### #    ####   #  ###  #  # ##                                     
 #### #  # #    #  #  #   #    ###  # #                                    
 #  # #  # #  # #  # #    #    #    # #  #                                 
 #  # #  #  ##  #  # #### #### #    #  #                                   

In [79]:
r

                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
       ##  #  #  ## 
      #  # #  # #  #
      #  # #### #   
      #### #  # #   
      #  # #  # #  #
      #  # #  #  ## 

In [12]:
xs, ys = zip(*r.panel.keys())

In [16]:
min(xs), max(xs), min(ys), max(ys)

(-1, 1162, -2, 1161)

In [ ]:
AHCHZEPK